In [1]:
import psycopg2
from psycopg2.extensions import register_adapter, AsIs

In [2]:
def select(register,complexType='r',dbname="mimic"):
    conn = psycopg2.connect("dbname="+dbname)
    cur = conn.cursor()
    if complexType=='r':
        select_stament="Sample ="+str(register['s_from'])
    else :
        select_stament="Sample BETWEEN "+str(register['s_from'])+" AND "+str(register['s_to'])
    select_stament = "SELECT sample,amp FROM "+complexType+" WHERE record='"+register['record']+"' AND "+select_stament+" LIMIT 1"
    cur.execute(select_stament)
    for row in cur :
        register[complexType+'_s']=row[0]
        register[complexType+'_a']=row[1]
    conn.close()
    return register

In [3]:
def insertRow(row,dbname="mimic") :
    conn = psycopg2.connect("dbname="+dbname)
    cur = conn.cursor()
    insert_statement = 'INSERT into rstq (%s) values %s'
    columns = row.keys()
    values = [row[column] for column in columns]
#    print(cur.mogrify(insert_statement, (AsIs(','.join(columns)), tuple(values))))
    cur.execute(insert_statement, (AsIs(','.join(columns)), tuple(values)))
    conn.commit()
    conn.close()

In [4]:
def difference(first,second):
    if first is None or second is None :
        return None
    else :
        return first-second

In [5]:
def readHeartBeats(record,dbname="mimic") :
    conn = psycopg2.connect("dbname="+dbname)
    cur = conn.cursor()
    select_stament = "SELECT s_from,s_to FROM heartbeatstemporal WHERE s_to - s_from < 1000 AND record ='"+record+"' "
    cur.execute(select_stament)
    print(record)
    for row in cur :
        register = {'record':record,'s_from':row[0],'s_to':row[1]}
        register = select(register)
        register = select(register,complexType='s')
        register = select(register,complexType='t')
        register = select(register,complexType='q')
        register['qt'] = difference(register.get('q_s'),register.get('t_s'))
        register['ts'] = difference(register.get('t_s'),register.get('s_s'))
        register['sr'] = difference(register.get('s_s'),register.get('r_s'))
        del register['s_from']
        del register['s_to']
        insertRow(register)
    conn.close()
#        selectrow(record[0])

In [ ]:
def distinctRecord(dbname="mimic"):
    conn = psycopg2.connect("dbname="+dbname)
    cur = conn.cursor()
    select_stament = "SELECT DISTINCT r.record FROM r where record not in (select distinct record from rstq) "
    cur.execute(select_stament)
    for record in cur :
        readHeartBeats(record[0])

In [ ]:
distinctRecord()

mimic2wdb/matched/s18108/s18108-3167-07-18-14-48
mimic2wdb/matched/s14995/s14995-2717-08-29-17-06
mimic2wdb/matched/s18402/s18402-2687-06-03-17-53
mimic2wdb/matched/s23552/s23552-2951-10-14-11-47
mimic2wdb/matched/s04481/s04481-2991-10-24-17-02
mimic2wdb/matched/s04317/s04317-2599-11-11-11-32
mimic2wdb/matched/s04331/s04331-2696-12-22-07-41
mimic2wdb/matched/s14266/s14266-2770-01-13-15-29
mimic2wdb/matched/s14947/s14947-3433-11-06-12-07
mimic2wdb/matched/s18970/s18970-3262-02-15-18-12
mimic2wdb/matched/s15302/s15302-2900-12-01-00-45
mimic2wdb/matched/s04324/s04324-3333-09-04-15-16
mimic2wdb/matched/s10651/s10651-3043-07-23-09-59
mimic2wdb/matched/s11242/s11242-2813-10-08-07-22
mimic2wdb/matched/s09366/s09366-3078-03-30-21-08
mimic2wdb/matched/s03995/s03995-3324-03-23-08-49
mimic2wdb/matched/s04806/s04806-2846-07-20-20-06
mimic2wdb/matched/s11744/s11744-2720-11-13-23-41
mimic2wdb/matched/s14204/s14204-2976-10-23-17-00
mimic2wdb/matched/s24228/s24228-2965-03-17-17-00
mimic2wdb/matched/s1